<a href="https://colab.research.google.com/github/MockaWolke/Places/blob/master/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
model_name = "ENB2-Aug-Transfer"

In [1]:
!git clone https://github.com/MockaWolke/Places.git
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/UAL/View_Directions.zip View_Directions.zip 
!cp drive/MyDrive/UAL/data.csv Places/data.csv 
!unzip -q View_Directions.zip -d Places
!rm  View_Directions.zip

import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import os

os.chdir("Places")
!mkdir models
import ds_generator
import models
import plotting

Cloning into 'Places'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 23 (delta 10), reused 20 (delta 7), pack-reused 0
Unpacking objects: 100% (23/23), done.
Mounted at /content/drive


In [2]:
train_ds, val_ds = ds_generator.get_train_and_val(name=model_name)
model = models.transfer_model()

31801344/31790344 [==============================] - 0s 0us/step


In [7]:
model_loc = "models/"+ model_name
log_dir = model_loc + "/logs"
checkpoint_path = model_loc + "/saves"
last_model = model_loc + "/last_model"

if model_name not in os.listdir("models"):

  os.makedirs(model_loc )
  os.makedirs(log_dir)
  os.makedirs(checkpoint_path)
  os.makedirs(last_model)
  


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.01,
    patience=10,
    verbose=1,
    mode='auto',
    restore_best_weights=True
) 

In [10]:
hist = model.fit(train_ds, epochs=25, validation_data=val_ds,callbacks=[tensorboard_callback,cp_callback,early_stopping_callback])

Epoch 1/25
1616/1616 [==============================] - ETA: 0s - loss: 0.5328 - accuracy: 0.8326
Epoch 1: val_loss improved from inf to 0.35080, saving model to models/ENB2-Start/saves
1616/1616 [==============================] - 485s 288ms/step - loss: 0.5328 - accuracy: 0.8326 - val_loss: 0.3508 - val_accuracy: 0.8692
Epoch 2/25
1616/1616 [==============================] - ETA: 0s - loss: 0.3985 - accuracy: 0.8482
Epoch 2: val_loss improved from 0.35080 to 0.34906, saving model to models/ENB2-Start/saves
1616/1616 [==============================] - 463s 286ms/step - loss: 0.3985 - accuracy: 0.8482 - val_loss: 0.3491 - val_accuracy: 0.8686
Epoch 3/25
1616/1616 [==============================] - ETA: 0s - loss: 0.3972 - accuracy: 0.8470
Epoch 3: val_loss did not improve from 0.34906
1616/1616 [==============================] - 463s 286ms/step - loss: 0.3972 - accuracy: 0.8470 - val_loss: 0.3930 - val_accuracy: 0.8554
Epoch 4/25
1616/1616 [==============================] - ETA: 0s - lo

In [11]:
plotting.plot_hist(hist)

In [14]:
model.save(last_model)

INFO:tensorflow:Assets written to: models/ENB2-Start/assets


INFO:tensorflow:Assets written to: models/ENB2-Start/assets


In [22]:
cp -a models/. ../drive/MyDrive/UAL/models/